In [11]:
import os
import pandas as pd
import csv
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
english_words_list = []

with open('english_words_corpora/english_corpus.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    list_of_csv = list(csv_reader)
    english_words_list = list_of_csv[0]      
english_words_set = set(english_words_list)


def create_relative_english_frequency_col(df):
    def calculate_relative_frequency(content):
        english_word_bools = [lemmatizer.lemmatize(word) in english_words_set for word in str(content).split() if word.isalpha()]
        if not english_word_bools:
            return 0
        return english_word_bools.count(True)/len(english_word_bools) 
    df["english_relative_frequency"] = df.cleaned_content.apply(calculate_relative_frequency)
    return df

def filter_by_english_thresholds(min_thresh, max_thresh, original_dataset, create_file = False):
    output_dir = os.path.splitext(original_dataset)[0]
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    with open(original_dataset, "r") as infile:
        source_df = pd.read_csv(infile, index_col=[0])
        out_df = create_relative_english_frequency_col(source_df)
        out_df = out_df[(min_thresh <= out_df["english_relative_frequency"]) & (out_df["english_relative_frequency"] < max_thresh)]
    
    if create_file: 
        with open(output_dir + f"/{min_thresh}_to_{max_thresh}_english_words.csv", "w") as outfile:
            outfile.write(out_df.to_csv())
            
    return out_df

In [12]:
for i in range(10):
    filter_by_english_thresholds(i/10, (i+1)/10, "data/Singapore/tweets_over_period/24400_tweets_over_period.csv", create_file = True)
filter_by_english_thresholds(1, 1.01, "data/Singapore/tweets_over_period/24400_tweets_over_period.csv")

,id,date,rawContent,cleaned_content,english_relative_frequency
7,8,2022-01-01 23:57:20+00:00,@Siennafrst I survived another year without tr...,i survived another year without travel in the ...,1.0
8,9,2022-01-01 23:56:08+00:00,#とき宣お正月 #超ときめき宣伝部 \n??? https://t.co/UZA8k8PSi9,NaN,1.0
9,10,2022-01-01 23:55:49+00:00,おはようございます😊\n今日は、これから息子の学校のペンキ塗りと荷物運び❗\nお正月ってどこ...,NaN,1.0
12,13,2022-01-01 23:52:10+00:00,@afrorakda ありがとうございます🥰\n普通に遊べて楽しいじゃないか～と自画自賛して...,NaN,1.0
13,14,2022-01-01 23:52:01+00:00,@Khithitofficial လေးစားပါတယ်\nသန်း‌ေြခာက်ဆယ်သြ...,NaN,1.0
...,...,...,...,...,...
92,24193,2022-08-31 22:48:29+00:00,Breakfast (@ Beach Road Kitchen) https://t.co/...,breakfast beach road kitchen,1.0
93,24194,2022-08-31 22:47:16+00:00,All the replies are complaints (I think 8 is a...,all the replies are complaints i think 8 is a ...,1.0
95,24196,2022-08-31 22:44:03+00:00,@Mohamed08334779 Masha allah,masha allah,1.0
96,24197,2022-08-31 22:42:29+00:00,NOPE https://t.co/G4YfsotYO0,nope,1.0
